#Imports

In [ ]:
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
from torch import nn
import torch
import torch.optim as optim
from tqdm import tqdm

import numpy as np

#Hyperparameters

In [ ]:
BATCH_SIZE=64
LEARNING_RATE=0.01
EPOCHS=100
HIDDEN_SIZE=256
ACTIVATION_FUNCTION = nn.ReLU

#Data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

full_trainSet = MNIST(root='./data', train=True, download=True, transform=transform)

subset_size = int(0.5 * len(full_trainSet))

subset_indices = np.random.choice(len(full_trainSet), subset_size, replace=False)

subset_trainSet = Subset(full_trainSet, subset_indices)

trainLoader = DataLoader(subset_trainSet, batch_size=BATCH_SIZE, shuffle=True)

testSet = MNIST(root='./data', train=False, download=True, transform=transform)
testLoader = DataLoader(testSet, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
len(trainLoader)

469

#Models

##Sub Layers

In [ ]:
class MainLayer(nn.Module):
  def __init__(self, inputSize, activationFunction, outputSize):
    super().__init__()
    self.sequential = nn.Sequential(
      nn.Linear(inputSize, outputSize),
      activationFunction(),
    )

class InputLayer(MainLayer):
  def forward(self, input):
    return self.sequential(input)

class HiddenLayer(MainLayer):
  def forward(self, input):
    return self.sequential(input)

class OutputLayer(MainLayer):
  def forward(self, input):
    return self.sequential(input)


##Recognizer Model

In [ ]:
class RecognizerModel(nn.Module):
  def __init__(self, inputSize, hiddenSize, outputSize):
    super().__init__()
    self.sequential = nn.Sequential(
      InputLayer(inputSize, ACTIVATION_FUNCTION, hiddenSize),

      HiddenLayer(hiddenSize, ACTIVATION_FUNCTION, hiddenSize),
      HiddenLayer(hiddenSize, ACTIVATION_FUNCTION, hiddenSize),

      OutputLayer(hiddenSize, ACTIVATION_FUNCTION, outputSize)
    )

  def forward(self, input):
    flattenImage = input.flatten(start_dim=1)
    return self.sequential(flattenImage)

recognizerModel=RecognizerModel(784, HIDDEN_SIZE, 10)

#Optimizer & Loss

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(recognizerModel.parameters(), lr=LEARNING_RATE)

def parametersCount(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"model parameters: {parametersCount(recognizerModel):,}")

model parameters: 335,114


#Training

In [ ]:
class Train():
    def __init__(self, model: nn.Module, loss, optimizer, epochs: int, trainLoader: DataLoader, testLoader: DataLoader):
        self.model = model
        self.loss = loss
        self.optimizer = optimizer
        self.epochs = epochs
        self.trainLoader = trainLoader
        self.testLoader = testLoader
        self.startTraining()

    def startTraining(self):
        for epoch in range(self.epochs):
          trainLoss = self.train(epoch)
          self.test()

          print(f"Epoch [{epoch + 1}/{self.epochs}], Training Loss: {trainLoss.item()}")


    def train(self, epoch):
      self.model.train()
      for input, target in self.trainLoader:
        self.optimizer.zero_grad()

        output = self.model(input)
        loss = self.loss(output, target)
        loss.backward()

        self.optimizer.step()

      return loss

    def test(self):
      self.model.eval()
      with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for input, target in self.testLoader:
          output = self.model(input)
          _, predicted = torch.max(output, 1)
          total_correct += (predicted == target).sum().item()
          total_samples += target.size(0)

        accuracy = total_correct / total_samples * 100
        print(f"Test Accuracy: {accuracy:.2f}%")
Train(recognizerModel, loss, optimizer, EPOCHS, trainLoader, testLoader)


Test Accuracy: 73.21%
Epoch [1/100], Training Loss: 1.1847988367080688
Test Accuracy: 85.10%
Epoch [2/100], Training Loss: 0.4770813286304474
Test Accuracy: 89.05%
Epoch [3/100], Training Loss: 0.3564029633998871


KeyboardInterrupt: 